In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os
import json
import pickle
from typing import Optional

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import shap
from sklearn.model_selection import GroupKFold, StratifiedGroupKFold, PredefinedSplit
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from sklearn.preprocessing import LabelEncoder  
from dataclasses import dataclass
from tqdm import tqdm
from sklearn.metrics import matthews_corrcoef, confusion_matrix

matplotlib.rcParams['animation.embed_limit'] = 2**128
import matplotlib.animation as animation
from matplotlib import rc
rc('animation', html='jshtml')
from utils.general import LabelEncoders, LGBMSerializer, reduce_dtype
from utils.nfl import merge_tracking, distance, cast_player_id
from utils.metrics import binarize_pred, summarize_per_play_mcc
from utils.visualize import EDAConfig, plot_animation
from utils.visualize_helmet import save_separate_videos, save_concat_video

In [ ]:
TRAINED_DATA_PATH = "./"
tracking = reduce_dtype(cast_player_id(pd.read_csv("../input/nfl-player-contact-detection/train_player_tracking.csv")))
labels = cast_player_id(pd.read_csv("../input/nfl-player-contact-detection/train_labels.csv"))
serializer = LGBMSerializer.from_file(os.path.join(TRAINED_DATA_PATH, "lgb"))
oof = np.load(os.path.join(TRAINED_DATA_PATH, "oof.npy"))
x_train = np.load(os.path.join(TRAINED_DATA_PATH, "X_train.npy"))

labels = merge_tracking(
    labels,
    tracking,
    ["x_position", "y_position"]
)
labels["distance"] = distance(labels["x_position_1"], labels["y_position_1"], labels["x_position_2"], labels["y_position_2"])
is_hard_sample = np.logical_or(labels["distance"]<=3, labels["nfl_player_id_2"] == -1)

assert is_hard_sample.sum() == len(oof)

is_ground = (labels[is_hard_sample]["nfl_player_id_2"] == -1).values
y_pred = binarize_pred(oof, serializer.threshold_1, serializer.threshold_2, is_ground)

labels.loc[is_hard_sample, "oof"] = oof
labels.loc[is_hard_sample, "y_pred"] = y_pred
labels['error'] = (labels['contact'] - labels['oof'].fillna(0.0)).abs()

per_play_mcc_df = summarize_per_play_mcc(labels)
per_play_mcc_df[per_play_mcc_df["number_of_contacts"]>0].sort_values(by="mcc").head(20)

helmet = pd.read_csv('../input/nfl-player-contact-detection/train_baseline_helmets.csv')

In [12]:
labels

,contact_id,game_play,datetime,step,nfl_player_id_1,nfl_player_id_2,contact,x_position_1,y_position_1,x_position_2,y_position_2,distance,oof,y_pred,error
0,58168_003392_0_38590_43854,58168_003392,2020-09-11T03:01:48.100Z,0,38590,43854,0,40.330002,25.280001,41.990002,16.790001,8.650763,NaN,NaN,0.000000e+00
1,58168_003392_0_38590_41257,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41257,0,40.330002,25.280001,45.770000,15.590000,11.112592,NaN,NaN,0.000000e+00
2,58168_003392_0_38590_41944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,41944,0,40.330002,25.280001,42.000000,22.850000,2.948524,4.270992e-07,False,4.270992e-07
3,58168_003392_0_38590_42386,58168_003392,2020-09-11T03:01:48.100Z,0,38590,42386,0,40.330002,25.280001,45.869999,23.889999,5.711714,NaN,NaN,0.000000e+00
4,58168_003392_0_38590_47944,58168_003392,2020-09-11T03:01:48.100Z,0,38590,47944,0,40.330002,25.280001,42.099998,26.580000,2.196107,1.000177e-05,False,1.000177e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4721613,58582_003121_91_48220_G,58582_003121,2021-10-12T02:42:29.100Z,91,48220,-1,0,33.180000,25.260000,NaN,NaN,NaN,6.953448e-04,False,6.953448e-04
4721614,58582_003121_91_47906_G,58582_003121,2021-10-12T02:42:29.100Z,91,47906,-1,0,38.250000,30.440001,NaN,NaN,NaN,2.232142e-04,False,2.232142e-04
4721615,58582_003121_91_38557_G,58582_003121,2021-10-12T02:42:29.100Z,91,38557,-1,0,23.690001,19.500000,NaN,NaN,NaN,1.291791e-03,False,1.291791e-03
4721616,58582_003121_91_47872_G,58582_003121,2021-10-12T02:42:29.100Z,91,47872,-1,0,62.630001,42.980000,NaN,NaN,NaN,1.139169e-04,False,1.139169e-04


In [ ]:
def get_id_by_index(labels, index):
    row = labels.loc[index]
    return row.game_play, row.nfl_player_id_1, row.nfl_player_id_2

# GAME_PLAY = "58529_001853"  # Noneならランダムに選ぶ
# PLAYER_ID_1 = None  # intで指定。指定しない場合、GAME_PLAYからランダムに選ぶ。
# PLAYER_ID_2 = None  # intで指定(G:-1)。指定しない場合、GAME_PLAYからランダムに選ぶ。
CONTACT_TO_G = False  # player_id_2を指定しないとき、対Gを選択するかどうか

# index直指定
# GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2 = get_id_by_index(labels, 1089722)
# print(GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2)

# errorの平均で悪い順にみる
samples = labels.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])['error'].sum().reset_index().sort_values('error', ascending=False)
GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2, error_mean = samples.iloc[0].values
print(GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2, error_mean)

In [ ]:
def evel_pred(y_true, y_pred):
    results = {}
    results['mcc'] = matthews_corrcoef(y_true, y_pred)
    results['acc'] = (y_true == y_pred).mean()
    results['n_tp'] = ((y_true == y_pred) & (y_pred == 1)).sum()
    results['n_tn'] = ((y_true == y_pred) & (y_pred == 0)).sum()
    results['n_fp'] = ((y_true != y_pred) & (y_pred == 1)).sum()
    results['n_fn'] = ((y_true != y_pred) & (y_pred == 0)).sum()
    results['num'] = len(y_true)
    return results

from tqdm.auto import tqdm

all_results = []
for (game_play, p1, p2), single_labels in tqdm(labels.groupby(['game_play', 'nfl_player_id_1', 'nfl_player_id_2'])):
    y_true = single_labels['contact'].fillna(0).astype(int)
    y_pred = single_labels['y_pred'].fillna(0).astype(int)
    results = evel_pred(y_true, y_pred)
    results['game_play'] = game_play
    results['nfl_player_id_1'] = p1
    results['nfl_player_id_2'] = p2
    all_results.append(results)
result_df = pd.DataFrame(all_results)
result_df = result_df.sort_values('acc')

In [ ]:
result_df.to_csv('summay.csv', index=False)

In [11]:
def _format(k, v):
    if type(v) == int:
        return f'{k}:{v:<3}'
    else:
        return f'{k}:{v:.3f}'
        
for i in range(100):
    row_dict = result_df.iloc[i].to_dict()
    game_play = row_dict.pop('game_play')
    nfl_player_id_1 = row_dict.pop('nfl_player_id_1')
    nfl_player_id_2 = row_dict.pop('nfl_player_id_2')
    text = ' '.join([_format(k, v) for k, v in row_dict.items()])
    print(game_play, nfl_player_id_1, nfl_player_id_2)
    print(text)
    end_path, side_path = save_separate_videos(labels, helmet, game_play, nfl_player_id_1, nfl_player_id_2, text)

58187_002329 46241 -1
mcc:0.000 acc:0.109 n_tp:0   n_tn:7   n_fp:57  n_fn:0   num:64 



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


58403_001076 42765 -1
mcc:0.000 acc:0.220 n_tp:0   n_tn:13  n_fp:46  n_fn:0   num:59 



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


58551_003569 42375 53475
mcc:0.090 acc:0.287 n_tp:12  n_tn:11  n_fp:56  n_fn:1   num:80 


...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


58270_002527 38577 52480
mcc:0.000 acc:0.620 n_tp:0   n_tn:49  n_fp:30  n_fn:0   num:79 



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


58555_002563 45532 46527
mcc:0.000 acc:0.625 n_tp:0   n_tn:40  n_fp:24  n_fn:0   num:64 



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [ ]:
# if GAME_PLAY is None:
#     sample = labels[(labels["contact"]==1)&(labels["nfl_player_id_2"]==-1)].sample(1).iloc[0]
#     GAME_PLAY = sample.game_play

# config = EDAConfig(serializer, tracking, labels, 
#                    GAME_PLAY,
#                    x_train,
#                    is_hard_sample,
#                    player_id_1=PLAYER_ID_1,
#                    player_id_2=PLAYER_ID_2,
#                    to_g=CONTACT_TO_G)

# plot_animation(config)

In [ ]:
# concat_path = save_concat_video(labels, helmet, GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2)
# end_path, side_path = save_separate_videos(labels, helmet, GAME_PLAY, PLAYER_ID_1, PLAYER_ID_2)

In [ ]:
from IPython.display import Video

def play_video(video_path: str):
    frac = 1.25 # scaling factor for display
    display(
        Video(data=video_path, embed=True, height=int(720*frac), width=int(1280*frac))
    )
    
play_video(side_path)